In [3]:
!pip install --upgrade pip
!pip install numpy pandas matplotlib scikit-learn opencv-python pillow tqdm
!pip install tensorflow==2.14.0 # choose TF version compatible with your system
!pip install transformers sentencepiece # for pretrained NLP inference
!pip install streamlit # for final UI (optional)

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----------------------- ---------------- 1.0/1.8 MB 12.7 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 5.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: To modify pip, please run the following command:
C:\Users\adity\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [4]:
import os
os.makedirs('data/fer2013', exist_ok=True)
os.makedirs('data/music', exist_ok=True)
os.makedirs('models', exist_ok=True)
print('folders ready')

folders ready


In [6]:
import pandas as pd
fer_path = "C:\\Users\\adity\\Downloads\\fer2013.csv" # change
df_fer = pd.read_csv(fer_path)
print(df_fer.shape)
print(df_fer.columns)
print(df_fer['Usage'].value_counts())

(35887, 3)
Index(['emotion', 'pixels', 'Usage'], dtype='object')
Usage
Training       28709
PublicTest      3589
PrivateTest     3589
Name: count, dtype: int64


In [8]:
music_path = "C:\\Users\\adity\\Downloads\\spotify_millsongdata.csv" # your CSV with columns like artist,title,genre,tags,mood
music = pd.read_csv(music_path)
print(music.shape)
print(music.columns)
# create a meta_text column for recommendation later
cols_for_meta = ['artist','title','genre','tags','mood']
for c in cols_for_meta:
    if c not in music.columns:
        music[c] = ''
music['meta_text'] = music[cols_for_meta].astype(str).agg(' '.join, axis=1)
music.head()

(57650, 4)
Index(['artist', 'song', 'link', 'text'], dtype='object')


,artist,song,link,text,title,genre,tags,mood,meta_text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA...",,,,,ABBA
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen...",,,,,ABBA
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...,,,,,ABBA
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,,,,,ABBA
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,,,,,ABBA


In [ ]:
import numpy as np
import cv2
#converting pixels to array

def pixels_to_array(pixels_str, img_size=48):
    arr = np.fromstring(pixels_str, sep=' ', dtype=np.uint8)
    arr = arr.reshape((48,48))
    arr = cv2.resize(arr, (img_size,img_size))
    # replicate channel -> 3
    return np.stack([arr,arr,arr], axis=-1)


IMG_SIZE = 96
X = np.zeros((len(df_fer), IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8)
for i, px in enumerate(df_fer['pixels']):
    X[i] = pixels_to_array(px, IMG_SIZE)


y = df_fer['emotion'].values
print('X shape', X.shape)

X shape (35887, 96, 96, 3)


In [12]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, stratify=y, random_state=42)


train_datagen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1,
height_shift_range=0.1, horizontal_flip=True,
preprocessing_function=None)
val_datagen = ImageDataGenerator(preprocessing_function=None)


batch_size = 32
train_gen = train_datagen.flow(X_train, y_train, batch_size=batch_size)
val_gen = val_datagen.flow(X_val, y_val, batch_size=batch_size)

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2


num_classes = len(np.unique(y))
base = MobileNetV2(weights='imagenet', include_top=False, pooling='avg', input_shape=(IMG_SIZE,IMG_SIZE,3))
base.trainable = False


inputs = keras.Input(shape=(IMG_SIZE,IMG_SIZE,3))
x = base(inputs, training=False)
x = layers.Dropout(0.4)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)
model = keras.Model(inputs, outputs)


model.compile(optimizer=keras.optimizers.Adam(1e-3),
loss='sparse_categorical_crossentropy', metrics=['accuracy'])


model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_96             │ (None, 1280)           │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,588,743 (9.88 MB)

 Trainable params: 330,247 (1.26 MB)

 Non-trainable params: 2,258,496 (8.62 MB)

In [15]:
callbacks = [keras.callbacks.ModelCheckpoint('models/fer_tl.h5', save_best_only=True, monitor='val_accuracy'),
keras.callbacks.ReduceLROnPlateau(patience=3, factor=0.5, monitor='val_loss')]


# initial quick training
model.fit(train_gen, validation_data=val_gen, epochs=6, callbacks=callbacks)


# fine-tune: unfreeze top layers
base.trainable = True
for layer in base.layers[:100]:
    layer.trainable = False


model.compile(optimizer=keras.optimizers.Adam(1e-5),
loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_gen, validation_data=val_gen, epochs=8, callbacks=callbacks)

c:\Users\adity\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/6
954/954 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step - accuracy: 0.2261 - loss: 2.1357

954/954 ━━━━━━━━━━━━━━━━━━━━ 271s 272ms/step - accuracy: 0.2526 - loss: 1.9347 - val_accuracy: 0.3369 - val_loss: 1.6904 - learning_rate: 0.0010
Epoch 2/6
954/954 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accuracy: 0.2825 - loss: 1.7673

954/954 ━━━━━━━━━━━━━━━━━━━━ 179s 187ms/step - accuracy: 0.2862 - loss: 1.7567 - val_accuracy: 0.3449 - val_loss: 1.6695 - learning_rate: 0.0010
Epoch 3/6
954/954 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step - accuracy: 0.2955 - loss: 1.7411

954/954 ━━━━━━━━━━━━━━━━━━━━ 204s 213ms/step - accuracy: 0.2944 - loss: 1.7393 - val_accuracy: 0.3477 - val_loss: 1.6647 - learning_rate: 0.0010
Epoch 4/6
954/954 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.3014 - loss: 1.7294

...

954/954 ━━━━━━━━━━━━━━━━━━━━ 134s 140ms/step - accuracy: 0.3007 - loss: 1.7234 - val_accuracy: 0.3590 - val_loss: 1.6492 - learning_rate: 0.0010
Epoch 6/6
954/954 ━━━━━━━━━━━━━━━━━━━━ 141s 148ms/step - accuracy: 0.3050 - loss: 1.7218 - val_accuracy: 0.3497 - val_loss: 1.6538 - learning_rate: 0.0010
Epoch 1/8
954/954 ━━━━━━━━━━━━━━━━━━━━ 231s 228ms/step - accuracy: 0.2448 - loss: 1.8512 - val_accuracy: 0.2327 - val_loss: 3.2295 - learning_rate: 1.0000e-05
Epoch 2/8
954/954 ━━━━━━━━━━━━━━━━━━━━ 327s 343ms/step - accuracy: 0.2876 - loss: 1.7657 - val_accuracy: 0.2773 - val_loss: 2.0752 - learning_rate: 1.0000e-05
Epoch 3/8
954/954 ━━━━━━━━━━━━━━━━━━━━ 183s 191ms/step - accuracy: 0.3114 - loss: 1.7215 - val_accuracy: 0.3401 - val_loss: 1.6994 - learning_rate: 1.0000e-05
Epoch 4/8
954/954 ━━━━━━━━━━━━━━━━━━━━ 175s 183ms/step - accuracy: 0.3268 - loss: 1.6939 - val_accuracy: 0.3575 - val_loss: 1.6403 - learning_rate: 5.0000e-06
Epoch 5/8
954/954 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step - accuracy:

954/954 ━━━━━━━━━━━━━━━━━━━━ 163s 171ms/step - accuracy: 0.3365 - loss: 1.6721 - val_accuracy: 0.3642 - val_loss: 1.6133 - learning_rate: 5.0000e-06
Epoch 6/8
954/954 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - accuracy: 0.3413 - loss: 1.6651

954/954 ━━━━━━━━━━━━━━━━━━━━ 161s 169ms/step - accuracy: 0.3415 - loss: 1.6602 - val_accuracy: 0.3735 - val_loss: 1.5977 - learning_rate: 5.0000e-06
Epoch 7/8
954/954 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.3474 - loss: 1.6608

954/954 ━━━━━━━━━━━━━━━━━━━━ 159s 167ms/step - accuracy: 0.3497 - loss: 1.6469 - val_accuracy: 0.3785 - val_loss: 1.5866 - learning_rate: 5.0000e-06
Epoch 8/8
954/954 ━━━━━━━━━━━━━━━━━━━━ 0s 329ms/step - accuracy: 0.3534 - loss: 1.6424

954/954 ━━━━━━━━━━━━━━━━━━━━ 328s 344ms/step - accuracy: 0.3562 - loss: 1.6305 - val_accuracy: 0.3895 - val_loss: 1.5703 - learning_rate: 5.0000e-06


In [2]:


from transformers import pipeline


# Simple sentiment analysis (binary: POSITIVE / NEGATIVE)
sentiment_analyzer = pipeline("sentiment-analysis")


examples = [
"I feel okay, not too bad.",
    "The movie was interesting but also confusing.",
    "I'm excited but a little nervous.",
    "That food was fine, nothing special."
]


for text in examples:
    res = sentiment_analyzer(text)[0]
    print(f"Text: {text}\nPredicted: {res['label']} (score={res['score']:.2f})\n")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


Text: I feel okay, not too bad.
Predicted: POSITIVE (score=1.00)

Text: The movie was interesting but also confusing.
Predicted: NEGATIVE (score=0.99)

Text: I'm excited but a little nervous.
Predicted: NEGATIVE (score=0.82)

Text: That food was fine, nothing special.
Predicted: NEGATIVE (score=0.98)

